# Total precipitation mean, stdm and quantiles

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu
import seeps_utils as seeps

In [4]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [5]:
model_name = 'IFS'
lead_range = conf[model_name]['lead_range']
verif_lead = [120,]
leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, verif_lead)

In [22]:
path_verif = conf[model_name]['save_loc_verif']+'TP_stats_{:03d}h_{}.nc'.format(
    verif_lead[0], model_name)

path_verif_ERA5 = conf[model_name]['save_loc_verif']+'TP_stats_{:03d}h_{}.nc'.format(
    verif_lead[0], 'ERA5')

## Verification setup

In [13]:
# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]

# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')

In [14]:
variable_levels = {'total_precipitation': None,}
variable_levels_IFS = {'total_precipitation_6hr': None,}
rename_IFS_to_ERA5 = {'total_precipitation_6hr': 'total_precipitation'}

In [15]:
# ---------------------------------------------------------------------------------------- #
# RMSE compute
tp_collect = []
tp_target = []

for fn_ours in filename_OURS:
    # detect 00Z vs 12Z
    ini = int(fn_ours[-6:-4])
    
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = ds_ours.isel(time=ind_lead)
    ds_ours = vu.ds_subset_everything(ds_ours, variable_levels_IFS)
    ds_ours = ds_ours.rename(rename_IFS_to_ERA5)
    
    ds_ours_24h = vu.accum_6h_24h(ds_ours, ini)
    # ds_ours_24h = ds_ours_24h.compute()
    tp_collect.append(ds_ours_24h)
    
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time'])
    ds_target = vu.ds_subset_everything(ds_target, variable_levels)
    ds_target_24h = vu.accum_6h_24h(ds_target, ini)
    # ds_target_24h = ds_target_24h.compute()
    tp_target.append(ds_target_24h)
    
    
# Combine verif results
ds_all_24h = xr.concat(tp_collect, dim='time')
ds_all_target_24h = xr.concat(tp_target, dim='time')

In [16]:
ds_mean = ds_all_24h['total_precipitation'].mean(dim='time')
ds_std = ds_all_24h['total_precipitation'].std(dim='time')
ds_p95 = ds_all_24h['total_precipitation'].quantile(0.95, dim='time')

# Combine these into a new dataset
ds_summary = xr.Dataset(
    {
    'mean': ds_mean,
    'std': ds_std,
    'p95': ds_p95
})

In [ ]:
# ds_summary.to_netcdf(path_verif, mode='w')

In [19]:
ds_all_target_24h = ds_all_target_24h.chunk({'time': -1})

In [20]:
ds_mean_target = ds_all_target_24h['total_precipitation'].mean(dim='time')
ds_std_target = ds_all_target_24h['total_precipitation'].std(dim='time')
ds_p95_target = ds_all_target_24h['total_precipitation'].quantile(0.95, dim='time')

# Combine these into a new dataset
ds_summary_target = xr.Dataset(
    {
    'mean': ds_mean_target,
    'std': ds_std_target,
    'p95': ds_p95_target
})

In [27]:
path_verif_ERA5

'/glade/campaign/cisl/aiml/ksha/CREDIT_physics/VERIF/IFS/TP_stats_120h_ERA5.nc'

In [28]:
# ds_summary_target.to_netcdf(path_verif_ERA5, mode='w')